# Step1. Import all required libraries

In [3]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager


# Step2. Install Chrome Driver

In [5]:
#Install driver
opts=webdriver.ChromeOptions()
opts.headless=True

driver = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)


C:\Users\Admin\AppData\Local\Temp\ipykernel_27888\1125334409.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  opts.headless=True
[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 8.53MB/s]
C:\Users\Admin\AppData\Local\Temp\ipykernel_27888\1125334409.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install() ,options=opts)


# Step3. Specify seach URL

In [6]:
search_url = "https://www.google.com/search?q={s}&tbm=isch&hl=en&tbs=il:cl&rlz=1C1CHBD_enSG948SG948&sa=X&ved=0CAAQ1vwEahcKEwiIsK6cjJX-AhUAAAAAHQAAAAAQAw&biw=875&bih=719"
driver.get(search_url.format(s='images%20of%20annotated%20diagrams'))

# Step4.  Write a function to take cursor to end of the page

In [7]:
def scroll_to_end(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)#sleep_between_interactions   
        

# Step5. Write a function to get url of each Image

In [16]:
#no license issues
from selenium.webdriver.common.by import By

def getImageUrls(name,totalImgs,driver):
    search_url = "https://www.google.com/search?q={s}&tbm=isch&hl=en&tbs=il:cl&rlz=1C1CHBD_enSG948SG948&sa=X&ved=0CAAQ1vwEahcKEwiIsK6cjJX-AhUAAAAAHQAAAAAQAw&biw=875&bih=719"

    driver.get(search_url.format(s=name))

    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results =  driver.find_elements(By.XPATH,"//img[contains(@class,'rg_i Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from {results_start}:{totalResults}")
        
        for img in thumbnail_results[results_start:totalResults]:
            
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.add(actual_image.get_attribute('src'))
            
            img_count=len(img_urls)
            
            if img_count >= totalImgs:
                print(f"Found: {img_count} image links")
                break
            else:
                print("Found:", img_count, "looking for more image links ...")                
                load_more_button = driver.find_element_by_css_selector(".mye4qd")
                driver.execute_script("document.querySelector('.mye4qd').click();")
                results_start = len(thumbnail_results)
    return img_urls
        

# Step6. Write a function to download each Image

In [17]:

def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

# Step7. Write a function to save each Image in Destination directory

In [18]:

def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in list(searchNames):
        path=os.path.join(destDir,name)
        if not os.path.isdir(path):
            os.mkdir(path)
        print('Current Path',path)
        totalLinks=getImageUrls(name,totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = f"{i:150}.jpg"
                downloadImages(path,file_name,link)
            
searchNames=['Car','horses'] 
destDir=f'./Dataset2/'
totalImgs=5

saveInDestFolder(searchNames,destDir,totalImgs,driver)





Current Path ./Dataset2/Car
Found: 100 search results. Extracting links from 0:100


AttributeError: 'WebDriver' object has no attribute 'find_elements_by_css_selector'